In [1]:
## Utility to copy notebooks from one workspace to another, in an ad-hoc manner

# Imports
from firecloud import api as fapi
import json
import os
import pandas as pd
import csv
from io import StringIO
from google.cloud import storage
import re
import hashlib

# Set parameters
src_bucket = 'fc-9cd4583e-7855-4b5e-ae88-d8971cfd5b46'
tar_bucket = 'fc-secure-6b1ba7f7-38f8-44e3-96b9-961fe1569e55'
notebooks_to_include = ['biosample_transforms.ipynb', 'build_file_manifest.ipynb', 'diagnosis_transforms.ipynb', 'donor_transforms.ipynb', 'familymember_transforms.ipynb', 'file_transforms.ipynb', 'ingest_pipeline.ipynb', 'output_data_profiling.ipynb', 'sequencingactivity_transforms.ipynb', 'source_files_creation.ipynb', 'tdr_dataset_ingest.ipynb', 'transform_task.ipynb', 'variantcall_transforms.ipynb'] # Leave empty to copy all notebooks
notebooks_to_exclude = []

# Functions
def get_notebooks_list(source_bucket, nbs_to_include=[], nbs_to_exclude=[]):
    
    # Collect list of objects/blobs from bucket 
    obj_list = []
    storage_client = storage.Client()
    storage_bucket = storage_client.bucket(source_bucket, user_project='dsp-data-ingest')
    objects = list(storage_client.list_blobs(storage_bucket, prefix = 'notebooks'))
    
    # Loop through list of objects and append names to final list
    for obj in objects:
        notebook_name = obj.name.split('/')[1]
        if len(nbs_to_include) > 0:
            for entry in nbs_to_include:
                if entry == notebook_name:
                    obj_list.append(notebook_name)
        elif len(nbs_to_exclude) > 0:
            for entry in nbs_to_exclude:
                if entry == notebook_name:
                    obj_list.append(notebook_name)
        else:
            obj_list.append(notebook_name)
    return obj_list

#print(get_notebooks_list('fc-9cd4583e-7855-4b5e-ae88-d8971cfd5b46'))
#print(get_notebooks_list('fc-9cd4583e-7855-4b5e-ae88-d8971cfd5b46', ['file_transforms.ipynb']))

def copy_notebooks(source_bucket, target_bucket, notebook_list):
    source_path = 'gs://' + source_bucket + '/notebooks'
    target_path = 'gs://' + target_bucket + '/notebooks'
    for item in notebook_list:
        !gsutil cp $source_path/$item $target_path/ 2> stdout

# Run ad-hoc copy
nb_copy_list = get_notebooks_list(src_bucket, notebooks_to_include, notebooks_to_exclude)
copy_notebooks(src_bucket, tar_bucket, nb_copy_list)
